In [1]:
import numpy as np
import pandas as pd
import os, sys

In [2]:
input_path = '../data/twitter_data'
output_path = '../data/twitter_filtered'

In [29]:
def score(row):
    rts, reps, likes = map(float, [row['retweets_count'], row['replies_count'], row['likes_count']])
    rts = rts if rts != np.nan else 0
    reps = reps if reps != np.nan else 0
    likes = likes if likes != np.nan else 0
    score = 5 * rts + 4 * reps * 3 * likes
    row['score'] = score
    return row

In [30]:
def includes_covid(row):
    includes = False
    keywords = ['covid', 'covid19', 'covid-19', 'virus',
                'quarentena', 'quarantine', 'quarantena',
                'corona', 'corona19', 'corona virus',
                'coronavirus', 'coronavírus', 'coronavirus19',
                'coronavírus19', 'positive', 'positivi',
                'positivo', 'positives', 'positivos',
                'cases', 'casos', 'caso', 'coronabond']
    for keyword in keywords:
        c = keyword in row['tweet'].lower()
        includes = includes or c
    row['includes'] = includes
    return row

In [31]:
total_related = 0
total_unrelated = 0

In [ ]:
for filename in os.listdir(input_path):
    if (filename == 'China_chinaorgch.tsv'):
        df = pd.read_json(f'{input_path}/{filename}')
    else:
        df = pd.read_csv(f'{input_path}/{filename}', sep='\t', engine='python')
    df = df.apply(score, axis=1)
    df = df.apply(includes_covid, axis=1)
    
    related = len(df[df.includes].reset_index(drop=True))
    total_related += related / len(df)
    
    unrelated = len(df[~df.includes].reset_index(drop=True))
    total_unrelated = unrelated / len(df)

    print(filename, related / len(df), unrelated / len(df))

Italy_virgilio_it.tsv 0.5625 0.4375
UK_BBCNews.tsv 0.2969529085872576 0.7030470914127424
UK_Telegraph.tsv 0.4001850138760407 0.5998149861239593
UK_guardian.tsv 0.2611497248769186 0.7388502751230813
China_ChinaDaily.tsv 1.0 0.0
Italy_repubblica.tsv 0.12805320435308343 0.8719467956469166


In [ ]:
nfiles = len(os.listdir(input_path))
print('Avg Related', total_related / nfiles, '; Avg Unrelated', total_unrelated / nfiles)

In [33]:
os.listdir(input_path)

['Italy_virgilio_it.tsv',
 'UK_BBCNews.tsv',
 'UK_Telegraph.tsv',
 'UK_guardian.tsv',
 'China_ChinaDaily.tsv',
 'Italy_repubblica.tsv',
 'Brazil_Estadao.tsv',
 'US_nytimes.tsv',
 'Brazil_JornalOGlobo.tsv',
 'Brazil_JornaldoBrasil.tsv',
 'Italy_Corriere.tsv',
 'China_PDChina.tsv',
 'Italy_Libero_official.tsv',
 'US_huffpost.tsv',
 'US_cnn.tsv',
 'China_chinaorgcn.tsv',
 'US_FoxNews.tsv',
 'China_shanghaidaily.tsv',
 'Brazil_folha.tsv',
 'UK_MailOnline.tsv',
 'Brazil_g1.tsv']

In [49]:
df = pd.read_json(f'{input_path}/{filename}')


In [51]:
df.head()

,id,datestamp,timestamp,username,tweet,replies_count,retweets_count,likes_count,url,country
0,1244502032523771904,2020-03-30,2020-03-30 02:49:40,chinaorgcn,[Video] Cloisonné - The 'Blue Jewel' in the ro...,1,1,8,https://twitter.com/chinaorgcn/status/12445020...,China
1,1244475851900465152,2020-03-30,2020-03-30 01:05:38,chinaorgcn,Opinion: Political considerations behind US co...,2,2,10,https://twitter.com/chinaorgcn/status/12444758...,China
2,1244445883879456768,2020-03-29,2020-03-30 23:06:34,chinaorgcn,Small businesses in Beijing seek to adapt amid...,2,6,32,https://twitter.com/chinaorgcn/status/12444458...,China
3,1244132958740475905,2020-03-29,2020-03-30 02:23:06,chinaorgcn,Wuhan deserves to be on the tourist bucket lis...,67,40,175,https://twitter.com/chinaorgcn/status/12441329...,China
4,1244126977126105089,2020-03-29,2020-03-30 01:59:20,chinaorgcn,Normal life in Wuhan gradually resumes https:...,9,21,67,https://twitter.com/chinaorgcn/status/12441269...,China
